# Treinamento de Modelos - Risco de Crédito

Este notebook realiza o treinamento de modelos de classificação e avalia a performance com métricas padrão.

In [1]:
%pip install pandas
%pip install joblib
%pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   --------- ------------------------------ 2.6/11.1 MB 13.8 MB/s eta 0:00:01
   -------------------- ------------------- 5.8/11.1 MB 14.1 MB/s eta 0:00:01
   ---------------------------- ----------- 7.9/11.1 MB 12.7 MB/s eta 0:00:01
   -------------------------------------- - 10.7/11.1 MB 13.2 MB/s eta 0:00:01
   ---------------------------------------- 11.1/11.1 MB 12.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/41.0 MB ? eta -:--:--
   - -------------------------------------- 1.8/41.0 MB 14.3 MB/s eta 0:00:03
   -- ------------------------------------- 2.4/41.0 MB 6.7 MB/s eta 0:00:06
   --- ------------------------------------ 3.1/41.0 MB 4.9 MB/s eta 0:00:08
   --- ------------------------------------ 3.9/41.0 MB 4.7 MB/s eta 0:00:08
   ---- ----------------------------------- 4.7/41.0 MB 4.6 MB/s eta 0:00:08
   -


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
# Importando as bibliotecas necessárias
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
import os

# Carregar dados
dados = pd.read_csv('../dados/dados_credito_tratados.csv')

# Remover coluna ID
dados = dados.drop(columns=['ID'])

# Separar variáveis
X = dados.drop(columns=['inadimplente'])
y = dados['inadimplente']

# Dividir em treino/teste
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar os dados
scaler = StandardScaler()
X_treino = scaler.fit_transform(X_treino)
X_teste = scaler.transform(X_teste)

# Balanceamento de classes usando SMOTE
smote = SMOTE(random_state=42)
X_treino_balanceado, y_treino_balanceado = smote.fit_resample(X_treino, y_treino)

# Treinar os modelos
modelo_lr = LogisticRegression(solver='saga', penalty='l2', max_iter=10000)
modelo_lr.fit(X_treino_balanceado, y_treino_balanceado)

modelo_rf = RandomForestClassifier(n_estimators=100, random_state=42)
modelo_rf.fit(X_treino_balanceado, y_treino_balanceado)

# Avaliar os modelos com cross-validation
cv_scores_rf = cross_val_score(modelo_rf, X_treino, y_treino, cv=5, scoring='accuracy')
cv_scores_lr = cross_val_score(modelo_lr, X_treino, y_treino, cv=5, scoring='accuracy')

print(f'Média da Acurácia com Cross-Validation para Random Forest: {cv_scores_rf.mean():.4f}')
print(f'Média da Acurácia com Cross-Validation para Regressão Logística: {cv_scores_lr.mean():.4f}')

# Ajustar o threshold de decisão (aqui estamos ajustando para 0.3, mas você pode experimentar outros valores)
threshold = 0.3
y_pred_rf = (modelo_rf.predict_proba(X_teste)[:, 1] > threshold).astype(int)
y_pred_lr = (modelo_lr.predict_proba(X_teste)[:, 1] > threshold).astype(int)

# Avaliar os modelos
for nome, modelo, y_pred in zip(['Random Forest', 'Regressão Logística'], [modelo_rf, modelo_lr], [y_pred_rf, y_pred_lr]):
    y_prob = modelo.predict_proba(X_teste)[:, 1]
    print(f'\nModelo: {nome}')
    print(f'Acurácia: {accuracy_score(y_teste, y_pred):.4f}')
    print(classification_report(y_teste, y_pred))
    print(f'AUC-ROC: {roc_auc_score(y_teste, y_prob):.4f}')

# Criar diretório para salvar os modelos se não existir
os.makedirs('../resultados/modelos', exist_ok=True)

# Salvando os modelos e o escalonador
joblib.dump(modelo_rf, '../resultados/modelos/modelo_rf.pkl')
joblib.dump(modelo_lr, '../resultados/modelos/modelo_lr.pkl')
joblib.dump(scaler, '../resultados/modelos/escalonador.pkl')

# Salvando os nomes das colunas usadas no treinamento
joblib.dump(X.columns, '../resultados/modelos/colunas_treinamento.pkl')


Média da Acurácia com Cross-Validation para Random Forest: 0.8161
Média da Acurácia com Cross-Validation para Regressão Logística: 0.8100

Modelo: Random Forest
Acurácia: 0.6682
              precision    recall  f1-score   support

           0       0.89      0.66      0.76      4687
           1       0.36      0.70      0.48      1313

    accuracy                           0.67      6000
   macro avg       0.63      0.68      0.62      6000
weighted avg       0.77      0.67      0.70      6000

AUC-ROC: 0.7507

Modelo: Regressão Logística
Acurácia: 0.3850
              precision    recall  f1-score   support

           0       0.89      0.24      0.38      4687
           1       0.25      0.89      0.39      1313

    accuracy                           0.39      6000
   macro avg       0.57      0.57      0.38      6000
weighted avg       0.75      0.39      0.38      6000

AUC-ROC: 0.7323


['../resultados/modelos/colunas_treinamento.pkl']